Устанавливаем необходимые библиотеки.

In [1]:
!pip install python-dotenv

  Using cached python_dotenv-0.19.2-py2.py3-none-any.whl (17 kB)


In [2]:
!pip install pymongo

  Using cached pymongo-4.0.1-cp37-cp37m-win_amd64.whl (354 kB)


In [3]:
!pip install "pymongo[srv]"

  Using cached dnspython-2.1.0-py3-none-any.whl (241 kB)


In [1]:
!pip install --upgrade srsparser

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for srsparser, since package 'wheel' is not installed.
  Attempting uninstall: srsparser
    Found existing installation: srsparser 1.0.7
    Uninstalling srsparser-1.0.7:
      Successfully uninstalled srsparser-1.0.7
    Running setup.py install for srsparser: started
    Running setup.py install for srsparser: finished with status 'done'


Подключаемся к базе данных с "распаршенными" текстовыми документами с ТЗ на MongoDB.

In [3]:
from pymongo import MongoClient
from dotenv import load_dotenv
import os

load_dotenv()

# строка подключения к кластеру MongoDB
CONNECTION_STRING = os.getenv('CONNECTION_STRING')

# название базы данных с результатами анализа текстовых документов
DB_NAME = "documentsAnalysis"
# название коллекции с результатами анализа текстовых документов
TMPL_COLL_NAME = "requirementsSpecifications"

# подключение к MongoDB
client = MongoClient(CONNECTION_STRING)
# подключение к БД на MongoDB
db = client[DB_NAME]
# подключение к коллекции, в которой находятся результаты анализа текстовых документов
coll = db[TMPL_COLL_NAME]
# взятие оттуда всех результатов
results = list(coll.find({}))

"Распаршенные" текстовые документы, где каждый отдельно взятый результат содержит название текстового документа с ТЗ и заполненную информацией структуру дерева разделов.

In [4]:
for result in results:
    print("Название документа:", result["document_name"], "\nРезультирующая структура:", result["structure"], end="\n\n")

Название документа: tz_01 
Результирующая структура: {'name': 'Техническое задание', 'children': [{'text': 'полное наименование системы и ее условное обозначение: Конфигурация «Бухгалтерия предприятия» в среде «1С: Предприятие 8.1»; \nшифр темы: ДП – УлГТУ –  08080165 – 00/00000 – 2010;\nнаименование предприятий разработчика и заказчика системы, их реквизиты: УМКУП «Городской градостроительный сервис», .Ульяновск, пер.Комсомольский 3, 8-962-633-01-77;\nперечень документов, на основании которых создается информационная система: ; \nплановые сроки начала и окончания работ:1 апреля . – 15 июня .;\nсведения об источниках и порядке финансирования работ: работа проводится на безвозмездной основе, не финансируется;\nпорядок оформления и предъявления заказчику результатов работ по созданию системы, ее частей и отдельных средств: по окончанию работ заказчику передается установочный диск со всеми дистрибутивами, необходимыми для функционирования разработанного программного продукта, а так же соп

Используя функцию get_tf_idf_weights из пакета srsparser, построим для каждого документа по всему тексту модель TF-IDF.

In [5]:
from srsparser import get_tf_idf_weights

# вытаскиваем из каждого резульата структуру дерева разделов
structs = []
for result in results:
    structs.append(result["structure"])

# для текстового содержания каждой структуры получаем TF-IDF веса слов
for struct_tf_idf_weights in get_tf_idf_weights(structs):
    print(struct_tf_idf_weights, end="\n\n") # вывод результатов в том же порядке, в каком результаты лежат в коллекции MongoDB

[['бухгалтерский', 0.174], ['полнота', 0.153], ['содержаться', 0.153], ['pack', 0.119], ['service', 0.119], ['базовой', 0.119], ['имеющей', 0.119], ['используемои', 0.119], ['полномочие', 0.119], ['помимо', 0.119], ['специфический', 0.119], ['заработной', 0.118], ['отображение', 0.118], ['современный', 0.118], ['этот', 0.118], ['восстановление', 0.114], ['экранный', 0.114], ['плата', 0.098], ['сумма', 0.093], ['возлагаться', 0.092], ['выдавать', 0.092], ['начисление', 0.092], ['неверный', 0.092], ['около', 0.092], ['путём', 0.092], ['разработанноить', 0.092], ['реакция', 0.092], ['этой', 0.092], ['эффективный', 0.092], ['рекомендоваться', 0.086], ['sql', 0.084], ['бухгалтерия', 0.084], ['поддержка', 0.084], ['предъявляться', 0.084], ['механизм', 0.083], ['принцип', 0.083], ['подсистема', 0.079], ['выше', 0.077], ['жесткия', 0.076], ['субд', 0.076], ['формат', 0.076], ['специалист', 0.073], ['microsoft', 0.072], ['расчёт', 0.066], ['несанкционированный', 0.065], ['осуществлять', 0.065],

Используя функцию get_tf_idf_weights из пакета srsparser, построим для каждого документа по разделу функциональные требования модель TF-IDF (Пустые массивы в выводе свидетельствуют о том, что в структуре не было раздела "Требования к функциям (задачам)").

In [7]:
# для текстового содержания раздела "Требования к функциям (задачам)" каждой структуры получаем TF-IDF веса слов
for struct_tf_idf_weights in get_tf_idf_weights(structs, "Требования к функциям (задачам)"):
    print(struct_tf_idf_weights, end="\n\n") # вывод результатов в том же порядке, в каком результаты лежат в коллекции MongoDB

[['бухгалтерский', 0.305], ['оперативный', 0.305], ['подсистема', 0.289], ['основа', 0.238], ['удаление', 0.184], ['учёт', 0.172], ['элемент', 0.163], ['список', 0.152], ['иметь', 0.13], ['отчёт', 0.13], ['просмотр', 0.13], ['документ', 0.126], ['вести', 0.119], ['возможно', 0.119], ['импорт', 0.119], ['итоговой', 0.119], ['лишь', 0.119], ['обладать', 0.119], ['объект', 0.119], ['ориентированный', 0.119], ['основной', 0.119], ['осуществлять', 0.119], ['первичный', 0.119], ['подготовка', 0.119], ['постоянный', 0.119], ['произвольный', 0.119], ['случай', 0.119], ['сортировка', 0.119], ['ссылаться', 0.119], ['существующий', 0.119], ['сформировать', 0.119], ['удаляемый', 0.119], ['фильтрация', 0.119], ['функциональность', 0.119], ['экспорт', 0.119], ['этот', 0.119], ['аналитический', 0.092], ['входящий', 0.092], ['добавление', 0.092], ['заработной', 0.092], ['начисление', 0.092], ['новый', 0.092], ['плата', 0.092], ['поиск', 0.092], ['следующей', 0.092], ['соответствующей', 0.092], ['должн

Используя функцию get_tf_idf_weights из пакета srsparser, построим для каждого документа по всему тексту модель TF-IDF, но теперь только для СУЩЕСТВИТЕЛЬНЫХ.

In [8]:
# для текстового содержания каждой структуры получаем TF-IDF веса слов, исключив из текста все слова, кроме существительных
for struct_tf_idf_weights in get_tf_idf_weights(structs, part_of_speech="NOUN"):
    print(struct_tf_idf_weights, end="\n\n") # вывод результатов в том же порядке, в каком результаты лежат в коллекции MongoDB

[['полнота', 0.231], ['используемои', 0.179], ['полномочие', 0.179], ['отображение', 0.179], ['восстановление', 0.171], ['плата', 0.148], ['сумма', 0.14], ['начисление', 0.138], ['реакция', 0.138], ['бухгалтерия', 0.126], ['поддержка', 0.126], ['механизм', 0.126], ['принцип', 0.126], ['подсистема', 0.12], ['жесткия', 0.114], ['субд', 0.114], ['формат', 0.114], ['специалист', 0.11], ['расчёт', 0.1], ['поиск', 0.097], ['просмотр', 0.097], ['ситуация', 0.097], ['ос', 0.097], ['башаровой', 0.089], ['дополнение', 0.089], ['достаточность', 0.089], ['замена', 0.089], ['запрашиваемои', 0.089], ['идентификация', 0.089], ['импорт', 0.089], ['комсомольския', 0.089], ['лучшея', 0.089], ['масштабирование', 0.089], ['масштабируемость', 0.089], ['модернизация', 0.089], ['обрабатываемои', 0.089], ['пер', 0.089], ['сортировка', 0.089], ['указание', 0.089], ['умкупа', 0.089], ['участник', 0.089], ['фильтрация', 0.089], ['целостность', 0.089], ['экспорт', 0.089], ['элемент', 0.089], ['учёт', 0.085], ['тр

Используя функцию get_tf_idf_weights из пакета srsparser, построим для каждого документа по разделу функциональные требования модель TF-IDF (Пустые массивы в выводе свидетельствуют о том, что в структуре не было раздела "Требования к функциям (задачам)"), но теперь только для СУЩЕСТВИТЕЛЬНЫХ.

In [9]:
# для текстового содержания раздела "Требования к функциям (задачам)" каждой структуры получаем TF-IDF веса слов, исключив из текста все слова, кроме существительных
for struct_tf_idf_weights in get_tf_idf_weights(structs, "Требования к функциям (задачам)", "NOUN"):
    print(struct_tf_idf_weights, end="\n\n") # вывод результатов в том же порядке, в каком результаты лежат в коллекции MongoDB

[['подсистема', 0.407], ['основа', 0.335], ['удаление', 0.258], ['учёт', 0.241], ['элемент', 0.229], ['список', 0.214], ['отчёт', 0.182], ['просмотр', 0.182], ['документ', 0.178], ['импорт', 0.167], ['объект', 0.167], ['подготовка', 0.167], ['случай', 0.167], ['сортировка', 0.167], ['фильтрация', 0.167], ['функциональность', 0.167], ['экспорт', 0.167], ['добавление', 0.129], ['начисление', 0.129], ['плата', 0.129], ['поиск', 0.129], ['хранение', 0.121], ['расчёт', 0.107], ['сумма', 0.107], ['формирование', 0.106], ['система', 0.105], ['операция', 0.091], ['регистрация', 0.091], ['редактирование', 0.091], ['состав', 0.091], ['сотрудник', 0.091], ['процесс', 0.06], ['средство', 0.047], ['ввод', 0.036], ['работа', 0.026], ['функция', 0.026], ['информация', 0.022]]

[['заказ', 0.521], ['оплата', 0.337], ['регистрация', 0.333], ['выдача', 0.315], ['выработка', 0.204], ['сертификат', 0.204], ['стадия', 0.204], ['филиал', 0.204], ['цикл', 0.204], ['оформление', 0.157], ['подразделение', 0.157